In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import math

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [3]:
#1 a)

#in train.csv

train_means = df_train.groupby(['user_id'])['rating'].mean()
#print(train_means)

#in test.csv
test_means = df_test.groupby(['user_id'])['rating'].mean()
#print(test_means)



In [4]:
#1 b)

#in train.csv
train_item_means = df_train.groupby(['book'])['rating'].mean()
#print(train_item_means)

#in test.csv
test_item_means = df_test.groupby(['book'])['rating'].mean()
#print(test_item_means)


In [5]:
#1 c)

train_avgs = list(df_train['rating'])
test_avgs = list(df_test['rating'])

In [6]:
#1 c)
def rmse(arr1,arr2):
    return math.sqrt(mean_squared_error(arr1, arr2))

df_train_avg = df_train['rating'].mean()
df_test_avg = df_test['rating'].mean()

#print("Average value for ratings in train.csv = ",df_train_avg)
#print("Average value for ratings in test.csv = ",df_test_avg)

[df_train_avg]*len(df_train['rating'])
const_avg_train = [df_train_avg]*len(df_train['rating'])

[df_test_avg]*len(df_test['rating'])
const_avg_test = [df_test_avg]*len(df_test['rating'])

#in train.csv
rmse_train = rmse(train_avgs,const_avg_train)
print("Root mean square error for train.csv = " , rmse_train)


#in test.csv
rmse_test = rmse(test_avgs,const_avg_test)
print("Root mean square error for test.csv = " , rmse_test)

Root mean square error for train.csv =  1.781336407021028
Root mean square error for test.csv =  1.8370063986986185


In [12]:
#1 d)


#in train.csv
rows_train = df_train['user_id'].unique()
cols_train = df_train['book'].unique()
data = df_train[['user_id', 'book', 'rating']]
idict_train = dict(zip(cols_train, range(len(cols_train))))
udict_train = dict(zip(rows_train, range(len(rows_train))))
data.user_id = [ udict_train[i] for i in data.user_id ]
data['book'] = [ idict_train[i] for i in data['book'] ]
nmat_train = data.as_matrix()
#print(nmat_train)
naive_train = np.zeros((len(rows_train),len(cols_train)))
for row in nmat_train:
    naive_train[row[0],row[1]] = row[2]
    
print("Sparsity of train_matrix = ", np.count_nonzero(nmat_train==0)/np.prod(nmat_train.shape))


#in test.csv
rows_test = df_test['user_id'].unique()
cols_test = df_test['book'].unique()
data = df_test[['user_id', 'book', 'rating']]
idict_test = dict(zip(cols_test, range(len(cols_test))))
udict_test = dict(zip(rows_test, range(len(rows_test))))
data.user_id = [ udict_test[i] for i in data.user_id ]
data['book'] = [ idict_test[i] for i in data['book'] ]
nmat_test = data.as_matrix()
#print(nmat_test)
naive_test = np.zeros((len(rows_test),len(cols_test)))
for row in nmat_test:
    naive_test[row[0],row[1]] = row[2]
print("Sparsity of test_matrix = ", np.count_nonzero(nmat_test==0)/np.prod(nmat_test.shape))
#print(udict_test)

Sparsity of train_matrix =  0.015778357559945336
Sparsity of test_matrix =  0.006546644844517185


In [8]:
#1 e)
def predict_naive(user, item):
    
    prediction = imean[item] + umean[user] - amean
    return prediction

# train/test_means = umean ; train/test_item_means = imean ; df_train/test_avg = amean

def mean_fn(naive):
    amean1 = np.mean(naive[naive!=0])
    umean1 = sum(naive.T) / sum((naive!=0).T)
    umean1 = umean1[~np.isnan(umean1)]
    imean1 = sum(naive) / sum((naive!=0))
    imean1 = imean1[~np.isnan(imean1)]
    
    return amean1,umean1,imean1


#for train.csv

#naive = np.zeros((len(rows_train),len(cols_train)))
predictions =[]
targets = []

amean,umean,imean = mean_fn(naive_train)
for row in nmat_train:
    #naive[row[0], row[1]] = row[2]
    user, item, actual = row[0], row[1], row[2]
    predictions.append(predict_naive(int(user), int(item)))
    targets.append(actual)



rmse_train = rmse(targets,predictions)
print("Root mean square error for train.csv = " , rmse_train)




#for test.csv
#naive = np.zeros((len(rows_test),len(cols_test)))
amean,umean,imean = mean_fn(naive_test)
predictions =[]
targets = []
for row in nmat_test:
    #naive[row[0], row[1]] = row[2]
    user, item, actual = row[0], row[1], row[2]
    predictions.append(predict_naive(int(user), int(item)))
    targets.append(actual)

    

rmse_test = rmse(targets,predictions)
print("Root mean square error for test.csv = " , rmse_test)


Root mean square error for train.csv =  1.1444196071250665
Root mean square error for test.csv =  1.0777148094967743


In [9]:
#2 a)

#operating on test.csv

def cosine(vec1,vec2):
    return np.dot(vec1,vec2)/(np.linalg.norm(vec1) * np.linalg.norm(vec2))

def cos(mat, a, b):
    if a == b:
        return 1
    aval = mat.T[a].nonzero()
    bval = mat.T[b].nonzero()
    corated = np.intersect1d(aval, bval)
    if len(corated) == 0:
        return 0
    avec = np.take(mat.T[a], corated)
    bvec = np.take(mat.T[b], corated)
    val = 1 - cosine(avec, bvec)
    if np.isnan(val):
        return 0
    return val

def pr(mat, a, b, imean):
    if a == b:
        return 1
    aval = mat.T[a].nonzero()
    bval = mat.T[b].nonzero()
    corated = np.intersect1d(aval, bval)
    if len(corated) < 2:
        return 0
    avec = np.take(mat.T[a], corated)
    bvec = np.take(mat.T[b], corated)
    avec1 = avec - imean[a]
    bvec1 = bvec - imean[b]
    val = 1 - cosine(avec1, bvec1)
    if np.isnan(val):
        return 0
    return val

def itemsimilar(mat,option):
    #rows = data.user_id.unique()
    #cols = data['book'].unique()
    #naive = np.zeros((len(rows),len(cols)))
    amean ,umean , imean = mean_fn(mat)
    
    n = mat.shape[1]
    sim_mat = np.zeros((n,n))
    
    if option=='pr':
        for i in range(n):
            for j in range(n):
                sim_mat[i][j] = pr(mat, i, j, imean)
        sim_mat = (sim_mat + 1)/2
    elif option == 'cos':
        #print("COS")
        for i in range(n):
            for j in range(n):
                sim_mat[i][j] = cos(mat, i, j)
    else:
        #print("Default")
        sim_mat = cosine_similarity(mat.T)
    return sim_mat, amean, umean, imean
    

#cos_sim_5 = itemsimilar(nmat_test,'cos')
#[item, nzero].argsort()[::-1][:k]
#print(cos_sim_5)
 
def user_sim_cos(user,k):
    l =[]
    for i in range(nmat_test.shape()[0]):
        l.append(cos(nmat_test,user,i))
    
    l.remove(1)
    
    
    
    
    


In [10]:
#2 b)

def predict(user, item, mat, item_similarity, amean, umean, imean,k=5):
    nzero = mat[user].nonzero()[0]
    if len(nzero) == 0:
        return amean
    baseline = imean + umean[user] - amean
    choice = nzero[item_similarity[item, nzero].argsort()[::-1][:k]]
    
    prediction = ((mat[user, choice] - baseline[choice]).dot(item_similarity[item,choice])/ sum(item_similarity[item, choice])) + baseline[item]
    if np.isnan(prediction):
        prediction = amean
    if prediction > 10:
        prediction = 10
    if prediction < 1:
        prediction = 1
    return prediction
    


In [25]:
#2 c)

def get_results(train_data, test_data,option, k):
    # train_data.rename(columns={'user_id':'book','book':'user_id', 'rating':'rating'}, inplace = True)
    rows = train_data.user_id.unique()
    cols = train_data['book'].unique()
    train_data = train_data[['user_id', 'book', 'rating']]
    idict = dict(zip(cols, range(len(cols))))
    udict = dict(zip(rows, range(len(rows))))
    train_data.user_id = [ udict[i] for i in train_data.user_id ]
    train_data['book'] = [ idict[i] for i in train_data['book'] ]
    mat = train_data.as_matrix()
    full_mat = np.zeros((len(rows),len(cols)))
    for row in mat:
        full_mat[row[0], row[1]] = row[2]
    item_similarity, amean, umean, imean = itemsimilar(full_mat, option)
    preds=[]
    real=[]
    for i in mat:
        preds.append(predict(i[0], i[1], full_mat, item_similarity, amean, umean, imean, k=5))
        real.append(i[2])
    err1 = mean_squared_error(preds,real)
    print('Train Error')
    print('RMSE : %.4f' % err1)

    # test_data.rename(columns={'user_id':'book','book':'user_id', 'rating':'rating'}, inplace = True)
    rows = test_data.user_id.unique()
    cols = test_data['book'].unique()
    test_data = test_data[['user_id', 'book', 'rating']]
    idict = dict(zip(cols, range(len(cols))))
    udict = dict(zip(rows, range(len(rows))))
    test_data.user_id = [ udict[i] for i in test_data.user_id ]
    test_data['book'] = [ idict[i] for i in test_data['book'] ]
    mat = test_data.as_matrix()
    full_mat = np.zeros((len(rows),len(cols)))
    for row in mat:
        full_mat[row[0], row[1]] = row[2]
    item_similarity, amean, umean, imean = itemsimilar(full_mat, option)
    preds=[]
    real=[]
    for i in mat:
        preds.append(predict(i[0], i[1], full_mat, item_similarity, amean, umean, imean, k=5))
        real.append(i[2])
    err2 = mean_squared_error(preds,real)
    print('Test Error')
    print('RMSE : %.4f' % err2)
    '''
    rows = train_data['user_id'].unique()
    cols = train_data['book'].unique()
    train_data = train_data[['user_id', 'book', 'rating']]
    idict = dict(zip(cols, range(len(cols))))
    udict = dict(zip(rows, range(len(rows))))
    train_data.user_id = [ udict[i] for i in train_data.user_id ]
    train_data['book'] = [ idict[i] for i in train_data['book'] ]
    mat = train_data.as_matrix()
    full_mat = np.zeros((len(rows),len(cols)))
    for row in train_data:
        full_mat[row[0], row[1]] = row[2]
        
    item_similarity, amean, umean, imean = itemsimilar(full_mat, option)
    preds = []
    real =[]
    
    #predict(i[0], i[1], full_mat, item_similarity, amean, umean, imean, k=5)
    
    for row in train_data:
        user, item, actual = row[0], row[1], row[2]
        preds.append(predict(user, item,full_mat,item_similarity,amean,umean,imean,k=5))
        real.append(actual)
    
    err1 = rmse(real,preds)
    print('Train Error')
    print('RMSE : %.4f' % err1)
    
    rows = test_data['user_id'].unique()
    cols = test_data['book'].unique()
    test_data = test_data[['user_id', 'book', 'rating']]
    idict = dict(zip(cols, range(len(cols))))
    udict = dict(zip(rows, range(len(rows))))
    test_data.user_id = [ udict[i] for i in test_data.user_id ]
    test_data['book'] = [ idict[i] for i in test_data['book'] ]
    mat = test_data.as_matrix()
    full_mat = np.zeros((len(rows),len(cols)))
    
    item_similarity, amean, umean, imean = itemsimilar(full_mat, option)
    
    real = []
    preds = []
    
    for row in test_data:
        user, item, actual = row[0], row[1], row[2]
        preds.append(predict(user, item,full_mat,item_similarity,amean,umean,imean,k=5))
        real.append(actual)

    err2 = rmse(real,preds)
    print('Test Error')
    print('RMSE : %.4f' % err2)
    '''

#amean,umean,imean = mean_fn(naive_train)
#rows = df_train['user_id'].unique()
#cols = df_train['book'].unique()
get_results(pd.read_csv("train.csv"), pd.read_csv("test.csv"), "cos", 5)

    

Train Error
RMSE : 0.0543
Test Error
RMSE : 0.0060
